In [20]:
print("hello world")

hello world


In [21]:
a=[]
a.append(3)
a.append(4)
a.append(5)
print(a)

[3, 4, 5]


In [29]:
tek=[]
cift=[]
for i in a:
  if i % 2 == 0:
     cift.append(i)
  else:
     tek.append(i)

In [30]:
print(tek)
print(cift)

[3, 5]
[4]


In [31]:
tek.reverse()
print(tek)

[5, 3]


In [ ]:
# DENEME